In [1]:
!pip install -r ../requirements.txt
!pip install sklearn
!pip install tensorflow addons

  Using cached AddOns-0.7.zip (34 kB)
    ERROR: Command errored out with exit status 1:
     command: /home/studio-lab-user/.conda/envs/default/bin/python3.9 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-7vcizo02/addons_10f2db34e88a412c94ba1cb00a5f70d5/setup.py'"'"'; __file__='"'"'/tmp/pip-install-7vcizo02/addons_10f2db34e88a412c94ba1cb00a5f70d5/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-_izrymyc
         cwd: /tmp/pip-install-7vcizo02/addons_10f2db34e88a412c94ba1cb00a5f70d5/
    Complete output (8 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-7vcizo02/addons_10f2db34e88a412c94ba1cb00a5f70d5/setup.py", line 4, 

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
from collections import Counter

MAX_LENGTH = 200# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)

2022-03-29 08:16:59.402797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-29 08:16:59.402829: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Create the model

In [3]:
tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa",from_tf=True,)
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa",)

2022-03-29 08:17:08.747518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-29 08:17:08.747554: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-29 08:17:08.747577: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2022-03-29 08:17:08.750320: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers

# Load and transform the data

In [4]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train,y_train = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

In [5]:
# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='max_length', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Create and compile the model

In [6]:
input_ids_layer = tf.keras.layers.Input(shape=(200,), 
                                        name='input_ids', 
                                        dtype='int32')
input_attention_layer = tf.keras.layers.Input(shape=(200,), 
                                              name='input_attention', 
                                              dtype='int32')
last_hidden_state = model.roberta([input_ids_layer, input_attention_layer])[0]

one_more = tf.keras.layers.AveragePooling1D(1,200)(last_hidden_state)
pooling = tf.keras.layers.Reshape((768,))(one_more)
output = tf.keras.layers.Dense(13,activation='softmax')(pooling)
roberta = tf.keras.Model([input_ids_layer, input_attention_layer], output)

In [7]:
params = {'optimizer':tf.keras.optimizers.Adam(learning_rate=1e-4),
          'loss':tf.keras.losses.CategoricalCrossentropy(),
          'metrics':["accuracy"],# tf.keras.metrics.top_k_categorical_accuracy(k=3)
}
roberta.compile(**params)

In [8]:
roberta.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 200)]        0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 200)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124055040   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 200,                                           

# Created weights for each class and Train

In [9]:
counts = Counter(y_train)
weights = {i:1/j for i,j in counts.items()}

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath="./esign_checkpoint/",save_best_only=True,save_weights_only=False),
    tf.keras.callbacks.EarlyStopping(patience=2,)
]

roberta.fit(x=[X_train_ids,X_train_attention],
            y=pd.get_dummies(y_train),
            batch_size=32,
            epochs = 8,
            workers = -1,
            validation_split=0.2,
            callbacks = callbacks,
            class_weight = weights
           )

Epoch 1/8
  6/669 [..............................] - ETA: 4:28:37 - loss: 0.0013 - accuracy: 0.0990

In [20]:
# model.save_weights('./saved_weights.h5')

In [3]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = np.argmax(model.predict(X_train),axis = 1)
print(classification_report(y_train, y_pred))

ModuleNotFoundError: No module named 'sklearn'